## quest
- NSC_BND_M20 대상(from mongo)
- 목표변수 : 범주형, 설명변수 : 5개 선택
- train_test_split() 이용
- classification_report() 사용(각각 train, test set) 평가 의견 첨부
- option : NSC_BND_M20_preprocss.ipynb 이용가능

## 평가의견 
- accuracy_score(정확도) : 교육한 값(target_train)은 0.917 / 교육한 값을 토대로 시험한 값(target_test)은 0.923으로 예측률이 높음 / 데이터에 대한 예측이 의미가 있음
- precision_score(정밀도) : 0.9178 / 모델이 양성으로 예측한 케이스 중에서 실제로 양성인 비율이 높다는 것을 의미
- recall_score(재현율) : 0.9178 / 모델이 실제 양성 중에서 약 91.78%를 올바르게 감지하여 양성으로 예측했다는 것을 의미
- F1_SCORE : 0.9178/ 높은 값으로, 정밀도와 민감도의 균형을 이루었져 있음을 확인함.
- precision_score, recall_score , f1_score 이 모두 0.9178로 동일한 값 :  모델의 예측 결과의 불균형이 크지 않음 /즉, 모델이 양성과 음성 클래스를 비슷한 비율로 정확하게 예측함

    - 목적변수인 '

### 평가 수치에 관한 해석 
- accuracy_score(정확도) 값이 0.85보다 높아야 데이터의 의미가 있음
- 정밀도가 높을수록 모델이 양성으로 예측한 결과가 실제로 양성인 경우가 많다는 것을 나타냅니다. 양성 클래스에 대한 정밀도가 높다는 것을 나타냅니다.
- recall_score(재현율):  모델이 실제 양성 데이터를 놓치지 않고 잘 포착하는 능력
- "f1_score"정밀도와 민감도의 균형을 측정하는 지표
    - 양성(Positive)은 분석 대상이 되는 특정 클래스나 상황을 나타내며, 음성(Negative)은 해당 클래스나 상황이 아닌 것을 나타냅니다

## 지도학습  

#### 분류 - 다항분류
- 목적변수의 범주가 2개 이상인것 

In [59]:
import pymongo as mg
import pandas as pd 

In [60]:
client = mg.MongoClient(host='mongodb://localhost:27017')
database = client['db_NHIS'] #Database 선택
collection = database['Join Collection NSC2_BND_M20'] # collection 선택
cursor = collection.find({})
list_M20 = list(cursor) # casting
df_BND_M20 = pd.DataFrame(list_M20) 
df_BND_M20[:2]

,_id,RN_INDI,BTH_YYYY,DTH_YYYYMM,COD1,COD2,RN_KEY,RN_INST,MDCARE_STRT_DT,FORM_CD,...,OPRTN_YN,MDCARE_DD_CNT,VSHSP_DD_CNT,TOT_PRSC_DD_CNT,MCARE_RSLT_TYPE,EDC_ADD_RT,ED_RC_TOT_AMT,EDC_SBA,EDC_INSUR_BRDN_AMT,STD_YYYY
0,64e6bbca5b6c745c7e44d61f,294364,1921LE,200707.0,T08-T14,W00-W19,2002080298703,26452,20020830,3,...,0,1,1,3,5.0,0.15,10590,3000,7590,2002
1,64e6bbca5b6c745c7e44d620,294364,1921LE,200707.0,T08-T14,W00-W19,2002090466395,63715,20020910,3,...,0,1,1,2,5.0,0.15,16520,4950,11570,2002


#### 전처리
- 목표변수 : 'SICK_SYM1'
- 설명변수 : 'OPRTN_YN','TOT_PRSC_DD_CNT','ED_RC_TOT_AMT', 'EDC_SBA', 'EDC_INSUR_BRDN_AMT'

In [61]:
df_BND_M20['SICK_SYM1'].value_counts()

J209     8244
I10      4629
I109     3275
F_       3203
N_       3177
         ... 
H700        1
E058        1
M5397       1
T18         1
P289        1
Name: SICK_SYM1, Length: 3414, dtype: int64

In [62]:
len('F_'), len('J209')

(2, 4)

In [63]:
'J209'[1:], str('I109')[:3]

('209', 'I10')

In [64]:
def convertSICK_SYM(sick_sym) :
    if len(sick_sym) < 3 : 
        return None              ## 크기가 3보다 작은 null
    else : 
        return sick_sym[:3]          ## 3보다 크면 3자리에서 자름
    
## 3개를 자르는 이유는 대분류3자리 소분류 2자리로 나눠져있음 

In [65]:
convertSICK_SYM('J209'), convertSICK_SYM('F_')

('J20', None)

In [66]:
 df_BND_M20['SICK_SYM1_3'] = df_BND_M20['SICK_SYM1'].apply(convertSICK_SYM)

In [67]:
 df_BND_M20['SICK_SYM1_3'].value_counts()

J20    8917
I10    8427
J03    4187
M54    3904
J06    3460
       ... 
G97       1
K01       1
S28       1
I72       1
P28       1
Name: SICK_SYM1_3, Length: 747, dtype: int64

In [68]:
df_BND_M20_dropna = df_BND_M20.dropna()
df_BND_M20_dropna.isnull().sum()

_id                   0
RN_INDI               0
BTH_YYYY              0
DTH_YYYYMM            0
COD1                  0
COD2                  0
RN_KEY                0
RN_INST               0
MDCARE_STRT_DT        0
FORM_CD               0
MCARE_SUBJ_CD         0
SICK_SYM1             0
OFIJ_TYPE             0
OPRTN_YN              0
MDCARE_DD_CNT         0
VSHSP_DD_CNT          0
TOT_PRSC_DD_CNT       0
MCARE_RSLT_TYPE       0
EDC_ADD_RT            0
ED_RC_TOT_AMT         0
EDC_SBA               0
EDC_INSUR_BRDN_AMT    0
STD_YYYY              0
SICK_SYM1_3           0
dtype: int64

In [69]:
target_list = df_BND_M20_dropna['SICK_SYM1_3'].value_counts().index.to_list() ## 값에 따른 인텍스만 추출

In [70]:
target_list.index('L02'), target_list.index('J03')

(83, 6)

In [71]:
def applyIndexNumber(sick_sym_3) :
    inexNumber = target_list.index(sick_sym_3)
    return inexNumber

In [95]:
df_BND_M20_dropna['SICK_SYM1_3_target'].shape

(829,)

In [72]:
df_BND_M20_dropna['SICK_SYM1_3_target'] = df_BND_M20_dropna['SICK_SYM1_3'].apply(applyIndexNumber)
df_BND_M20_dropna[:2]

C:\Users\02-19\AppData\Local\Temp\ipykernel_8544\836018491.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_BND_M20_dropna['SICK_SYM1_3_target'] = df_BND_M20_dropna['SICK_SYM1_3'].apply(applyIndexNumber)


,_id,RN_INDI,BTH_YYYY,DTH_YYYYMM,COD1,COD2,RN_KEY,RN_INST,MDCARE_STRT_DT,FORM_CD,...,VSHSP_DD_CNT,TOT_PRSC_DD_CNT,MCARE_RSLT_TYPE,EDC_ADD_RT,ED_RC_TOT_AMT,EDC_SBA,EDC_INSUR_BRDN_AMT,STD_YYYY,SICK_SYM1_3,SICK_SYM1_3_target
0,64e6bbca5b6c745c7e44d61f,294364,1921LE,200707.0,T08-T14,W00-W19,2002080298703,26452,20020830,3,...,1,3,5.0,0.15,10590,3000,7590,2002,L02,83
3,64e6bbca5b6c745c7e44d622,294364,1921LE,200707.0,T08-T14,W00-W19,2002020488631,23506,20020208,3,...,1,3,5.0,0.15,12920,3000,9920,2002,K29,8


### 정형화

In [73]:
from sklearn.model_selection import train_test_split

In [74]:
target.value_counts()

6    829
Name: SICK_SYM1_3_target, dtype: int64

In [75]:
target = df_BND_M20_dropna['SICK_SYM1_3_target']
features = df_BND_M20_dropna[['OPRTN_YN','TOT_PRSC_DD_CNT','ED_RC_TOT_AMT', 'EDC_SBA', 'EDC_INSUR_BRDN_AMT']]
target.shape, features.shape

((829,), (829, 5))

In [76]:
features_train, features_test, target_train, target_test = train_test_split(features, target, random_state=111) #random_state=111 랜덤값으로 설정
features_train.shape, target_train.shape, features_test.shape, target_test.shape

((621, 5), (621,), (208, 5), (208,))

### 모델학습 

In [77]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier() # 인스턴스화
model.fit(features, target) #모델 훈련

DecisionTreeClassifier()

### 예측

### 평가 

In [78]:
target_train_predict = model.predict(features_train) 
target_train_predict.shape 

(621,)

In [79]:
from sklearn.metrics import accuracy_score

In [80]:
accuracy_score(target_train, target_train_predict) 
## 교육한값과 교육을 예측한 값과 비교(교내 시험)

0.9178743961352657

In [81]:
target_test_predict = model.predict(features_test) # test한 값의 예측값
target_test_predict.shape #target_test.shape과 동일

(208,)

In [82]:
accuracy_score(target_test, target_test_predict)

0.9230769230769231

#### f1

In [83]:
from sklearn.metrics import classification_report

In [84]:
print(classification_report(target_train, target_train_predict)) #교육

              precision    recall  f1-score   support

           0       0.76      0.96      0.85        53
           1       0.91      0.98      0.94        49
           2       0.91      1.00      0.95        30
           3       0.73      0.93      0.82        29
           4       1.00      1.00      1.00        24
           5       0.81      0.79      0.80        28
           6       0.95      0.77      0.85        26
           7       1.00      1.00      1.00        18
           8       1.00      0.85      0.92        13
           9       0.80      0.86      0.83        14
          10       1.00      1.00      1.00        11
          11       1.00      1.00      1.00        13
          12       1.00      1.00      1.00         8
          13       0.83      1.00      0.91        10
          14       0.88      1.00      0.93         7
          15       0.86      0.75      0.80         8
          16       0.88      0.78      0.82         9
          17       1.00    

C:\Users\02-19\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\02-19\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\02-19\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [85]:
print(classification_report(target_test, target_test_predict)) #시험

              precision    recall  f1-score   support

           0       0.74      0.93      0.82        15
           1       0.78      1.00      0.88         7
           2       1.00      0.95      0.97        20
           3       0.71      0.83      0.77         6
           4       1.00      1.00      1.00         8
           5       0.67      1.00      0.80         4
           6       0.80      1.00      0.89         4
           7       1.00      1.00      1.00         6
           8       1.00      0.90      0.95        10
           9       0.50      0.60      0.55         5
          10       1.00      1.00      1.00         5
          11       1.00      0.50      0.67         2
          12       1.00      1.00      1.00         4
          13       1.00      1.00      1.00         2
          14       1.00      1.00      1.00         4
          15       1.00      1.00      1.00         3
          16       0.67      1.00      0.80         2
          17       1.00    

C:\Users\02-19\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\02-19\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\02-19\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 오차 행렬 (confusion_matrix)
- precision : 정밀도 
- recall : 재현율

In [86]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

In [87]:
confusion_matrix(target_train, target_train_predict)

array([[51,  0,  0, ...,  0,  0,  0],
       [ 0, 48,  1, ...,  0,  0,  0],
       [ 0,  0, 30, ...,  0,  0,  0],
       ...,
       [ 0,  0,  0, ...,  1,  0,  0],
       [ 0,  0,  0, ...,  0,  1,  0],
       [ 0,  0,  0, ...,  0,  0,  1]], dtype=int64)

In [99]:
precision_score(target_train, target_train_predict, average='micro') #가중치?

0.9178743961352657

In [91]:
precision_score(target_test, target_test_predict, average='micro')

0.9230769230769231

In [92]:
recall_score(target_train, target_train_predict,average='micro')

0.9178743961352657

In [93]:
f1_score(target_train, target_train_predict,average='micro')

0.9178743961352658